# Planning report
## Create new tickets with fitting predictions easily

In [1]:
import ipywidgets as widgets
import pandas as pd
import plotly.express as px
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)


import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(colorscale='plotly', world_readable=True)

# Extra options
pd.options.display.max_rows = 30
pd.options.display.max_columns = 25

from datetime import date
from ipywidgets import interact, interact_manual
from src.db.utils import SnowflakeWrapper
from src.compute.utils import Interval
from src.compute.tickets import get_tickets
from src.compute.developer import get_developers, get_developer_ids, tickets_assigned_per_day
conn = SnowflakeWrapper.create_snowflake_connection()
sw = SnowflakeWrapper(conn)

In [2]:
#widgets.DatePicker(description='Pick a Date to', disabled=False)
interval = Interval(date(2019, 10, 1), date(2020, 1, 1))
df = get_tickets(sw, interval)

In [5]:
def plot_up_to(dev_id, date_from, date_to):
    current_interval = Interval(date_from, date_to)
    plot_df = tickets_assigned_per_day(sw, dev_id, current_interval)
    return px.histogram(
        plot_df,
        x="DAY_ASSIGNED",
        color="KEY",
        title=f"Issues on thee interval {interval})",
        hover_data=plot_df.columns
    )

_ = interact(
        plot_up_to,
        dev_id=widgets.widgets.Text(value='marko.prelevikj', placeholder='Developer ID', description='Choose Developer', disabled=False),
        sel=widgets.SelectMultiple(options=['Apples', 'Oranges', 'Pears'],value=['Oranges'],description='Options',disabled=False),
        date_from = widgets.DatePicker(value=date(2019,10,1)),
        date_to = widgets.DatePicker(value=date(2020,1,1))
    )

interactive(children=(Text(value='marko.prelevikj', description='Choose Developer', placeholder='Developer ID'…